# Time-series signatures from raw brdf imagery / smoothed ts outputs

In [ ]:
import os
import sys
from pathlib import Path
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
import shutil
import tempfile
import json
import random
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform
from pyproj import CRS
from shapely.geometry import box
from shapely.geometry import shape
from shapely.geometry import MultiPoint
from shapely.geometry import Point
from shapely.geometry import Polygon
from rasterio.plot import show
import xarray as xr

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from TSprofile import *

In [ ]:
##### Note: 
###  Notebook is housed on login node; time series calculations shold not be run on login node (and take a long time anyway). 
###  Better to install on cluster, run GettimeSeries.sh in Bash folder and copy output file path to parameters 
###  to read with `LoadTSfromFile(TSfile)` to load time series here for plotting.

###TODO: Fix bash script (and LUCinSA_helpers.py) so that it can take more than one cell at a time (cellList should be an array)
###TODO: Enable editing Bash script to run through Slurm via this notebook directly

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basicConfig
print("Run Parameters: \n raw_dir = {} \n gridCell = {} \n index_dir = {} \n gridFile = {} \n out_dir = {}"
      .format(basicConfig['raw_dir'],basicConfig['gridCell'],basicConfig['index_dir'],basicConfig['gridFile'],basicConfig['out_dir']))

%store -r TimeSeriesParams
print(" point_file = {}".format(TimeSeriesParams['PtFile']))
if TimeSeriesParams['LoadPreRunTS'] == True:
    print(" Smooth TS loaded from {}".format(TimeSeriesParams['SmoothTSfile']))
    print(" Raw TS loaded from {}".format(TimeSeriesParams['RawTSfile']))
    

In [ ]:
NatPts = GetPtsInGrid(basicConfig['gridFile'], basicConfig['gridCell'], TimeSeriesParams['PtFile'])
NatPts.head(n=5)

## Plot smooth time series

In [ ]:
if TimeSeriesParams['LoadPreRunTS'] == True:
    if TimeSeriesParams['SmoothTSfile']:
        smooth = LoadTSfromFile(TimeSeriesParams['SmoothTSfile'])
else:
    smooth = GetTimeSeriesForPts_MultiCell(basicConfig['out_dir'], basicConfig['spec_index'], TimeSeriesParams['StartYr'], TimeSeriesParams['EndYr'],
            basicConfig['smooth_dir'], 'Smooth', basicConfig['gridFile'], TimeSeriesParams['gridCells'], 
            groundPolys=TimeSeriesParams['groundPolys'], oldest=TimeSeriesParams['oldestSamp'], newest=TimeSeriesParams['newestSamp'],
            npts=TimeSeriesParams['npts'], seed=TimeSeriesParams['seed1'], loadSamp=TimeSeriesParams['loadSamp'], ptFile=TimeSeriesParams['PtFile'])

In [ ]:
smooth = smooth[smooth['ALL'] != 0]

fig = plt.figure(figsize=(10, 5))
plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
#plt.errorbar(raw.index, raw['ALL'], yerr=raw['stdv'], fmt='o', color='g')
plt.title('Smoothed {} for Yucca sample {}-{}'.format(basicConfig['spec_index'],TimeSeriesParams['StartYr'],TimeSeriesParams['EndYr']))
plt.ylim(0,7000)
plt.show();

## Plot raw time series

In [ ]:
if TimeSeriesParams['LoadPreRunTS'] == True:
    if TimeSeriesParams['RawTSfile']:
        raw = LoadTSfromFile(TimeSeriesParams['RawTSfile'])
else:
    raw = GetTimeSeriesForPts_MultiCell(basicConfig['out_dir'], basicConfig['spec_index'], TimeSeriesParams['StartYr'], TimeSeriesParams['EndYr'],
            basicConfig['raw_dir'], TimeSeriesParams['imageType'], basicConfig['gridFile'], TimeSeriesParams['gridCells'], 
            groundPolys=TimeSeriesParams['groundPolys'], oldest=TimeSeriesParams['oldestSamp'], newest=TimeSeriesParams['newestSamp'],
            npts=TimeSeriesParams['npts'], seed=TimeSeriesParams['seed1'], loadSamp=TimeSeriesParams['loadSamp'], ptFile=TimeSeriesParams['PtFile'])

In [ ]:
raw = raw[raw['ALL'] != 0]

fig = plt.figure(figsize=(10, 5))
#plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
plt.errorbar(raw.index, raw['ALL'], yerr=raw['stdv'], fmt='o', color='g')
plt.title('Raw {} for natural forest points {}-{}'.format(basicConfig['spec_index'], TimeSeriesParams['StartYr'],TimeSeriesParams['EndYr']))
plt.show()

## To plot a timeseries for each polygon in file:

In [ ]:
smooth = GetTimeSeriesForPts_MultiCell(basicConfig['out_dir'], basicConfig['spec_index'], TimeSeriesParams['StartYr'], TimeSeriesParams['EndYr'],
            basicConfig['smooth_dir'], 'Smooth', basicConfig['gridFile'], TimeSeriesParams['gridCells'], 
            groundPolys=TimeSeriesParams['groundPolys'], oldest=TimeSeriesParams['oldestSamp'], newest=TimeSeriesParams['newestSamp'],
            npts=TimeSeriesParams['npts'], seed=TimeSeriesParams['seed1'], loadSamp=TimeSeriesParams['loadSamp'], ptFile=TimeSeriesParams['PtFile'])

In [ ]:
smooth = smooth[smooth['ALL'] != 0]

fig = plt.figure(figsize=(10, 5))
plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
#plt.errorbar(raw.index, raw['ALL'], yerr=raw['stdv'], fmt='o', color='g')
plt.title('Smoothed {} for natural forest points {}-{}'.format(basicConfig['spec_index'],TimeSeriesParams['StartYr'],TimeSeriesParams['EndYr']))
plt.show();

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html

outName = str(basicConfig['country']+'2_TimeSeries'+str(basicConfig['gridCell'])+'_from_'+str(SinglePlotParams['plotYr'])+str(SinglePlotParams['plotDay']))
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$outName 1b_ExploreData_OpenImage.ipynb